In [ ]:
using Geodesy, PlotlyJS, Interact

In [ ]:
# lat is the latitude the shard is on, ang is the angle of the shard
function shard_area(lat, ang)
    a, b, c, d = [LatLon(lat, 0), LatLon(lat, ang), 
                  LatLon(lat+ang, ang), LatLon(lat+ang, 0)]
                    
    area = (distance(a,b)*distance(a,d))/1000000 # in km^2
    # sides approximated by cartesian distance
    # curvature should be neglegible at these sizes
    
    return(area)
end
    
# estimate population and reveals per day
function pop_reveals(area, pop_dens, inf_rate)
    pop = area*pop_dens
    reveals = pop*inf_rate
    return(pop, reveals)
end

# estimate bandwidth in MB per user and GB overall
function bandwidth(pop, reveals, report_size, rotations)
    bw_user = reveals * report_size * rotations
    bw_all = bw_user * pop
    return(bw_user/1000000, bw_all/1000000000)
end

In [ ]:
@manipulate for rmax in slider(1:1:48, value=12, label="max rotation"),
                amax in slider(0.05:0.05:0.5, label="max shard angle"),
                rs in slider(70:10:389, value=200, label="report size"),
                ir in slider([0.0001, 0.001, 0.01, 0.1], label="infection rate"),
                download = Dict("per user"=>1, "overall"=>0)

    layout = Layout(scene = attr(
                    xaxis_title = "rotations", 
                    yaxis_title = "angle (°)", 
                    zaxis_title = 
                        (if download == 1
                            "MB per user"
                         else
                            "GB overall"
                        end),
                    camera = attr(eye=attr(x=-1, y=-1.25, z=1))))
    style = Style(layout=layout)
    
        
    function bw(r,a)
        if download == 1
            b, _ = bandwidth(pop_reveals(shard_area(0,a), 6000, ir)..., rs, r)
        else
            _, b = bandwidth(pop_reveals(shard_area(0,a), 6000, ir)..., rs, r)
        end
        
        return(b)
    end

    rot = 1:1:rmax
    ang = 0.05:0.05:amax
    data = [bw(r,a) for r in rot, a in ang]

    user = surface(x=rot, y=ang, z=data)
    plot(user, style=style)
end
# sliders to explore different scales of the tradeoff. 
# (Camera gets reset on parameter change)